# Loading a saved model and merging the saved adapter weights
* this requires more memory than what the training takes up
* instance ml.g5.4xlarge
* make sure your libraries that are getting installed here match the ones you used at training stage

In [3]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops py7zr tensorboardX
!pip install -q -U torch==2.0.1 bitsandbytes==0.40.2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GPU Check

In [5]:
!nvidia-smi

Wed Nov  1 06:22:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Make sure the path is pointing to the correct checkpoint
* do not make any changes to the names of the files in the chek point direcotry

In [6]:
peft_model_id = "./results/checkpoint-150" # ensure this folder points to the right one
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True, 
    load_in_8bit=True, 
    device_map={"":0},
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)



Loading checkpoint shards: 100%|██████████| 2/2 [01:00<00:00, 30.25s/it]


In [7]:
! mkdir merged_model_and_adapter_weights

mkdir: cannot create directory ‘merged_model_and_adapter_weights’: File exists


In [8]:
model = model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:67: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [11]:
model.save_pretrained('./merged_model_and_adapter_weights')

In [12]:
# torch.save(model.state_dict(), './merged_model_and_adapter_weights/merged_peft_model.pt')

In [13]:
! ls -la ./merged_model_and_adapter_weights

total 7052452
drwxr-xr-x 3 root root       6144 Nov  1 06:26 .
drwxr-xr-x 5 root root       6144 Nov  1 06:26 ..
drwxr-xr-x 2 root root       6144 Nov  1 06:26 .ipynb_checkpoints
-rw-r--r-- 1 root root       1603 Nov  1 06:26 config.json
-rw-r--r-- 1 root root        113 Nov  1 06:26 generation_config.json
-rw-r--r-- 1 root root 7221688369 Nov  1 06:27 pytorch_model.bin
